In [1]:
import re,os
import pandas as pd
from natsort import natsorted
from Bio import SeqIO

In [2]:
hamster_id = '/data/shangzhong/Database/hamster/hamster_all_id.txt'
id_map_fn = '/data/shangzhong/Picr_assembly/Annotation/pasa_exonerate/01_gene_tr_pr.txt'
pasa_path = '/data/shangzhong/Picr_assembly/Annotation/PASA/pasa_rnaseq'
gmap_gff = pasa_path + '/gmap.spliced_alignments.gff3'
pasa_desp = pasa_path + '/picr_db_rna.pasa_assemblies_described.txt'
pasa_gff = pasa_path + '/picr_db_rna.pasa_assemblies.gff3'
transde_gff = pasa_path + '/picr_db_rna.assemblies.fasta.transdecoder.genome.gff3'
transde_cds = pasa_path + '/picr_db_rna.assemblies.fasta.transdecoder.pep'
exonerate_gff = '/data/shangzhong/Picr_assembly/Annotation/exonerate/exonerate.gff'
refseq_pr_fa = '/data/shangzhong/Picr_assembly/Annotation/hamster_pr.fa'

#### 1. Build gmap transcripts position.
* gmap_pos_dic = {tr:{chr+str:[[s1,e1],[s2,e2],...]}}
The reason why we build this one is that some transcripts in gmap maps to different chromosome. So we save them in different item in the dictionary.

In [3]:
# build gmap transcripts position
gmap_pos_dic = {}
with open(gmap_gff) as f:
    for line in f:
        if line.strip() == '' or line.startswith('#'):
            continue
        item = line.split('\t')
        chrom = item[0]
        start = int(item[3])
        end = int(item[4])
        anno = item[8]
        strand = item[6]
        tr = re.search('(?<=Target=).+?(?=\.)',anno).group(0)
        if tr not in gmap_pos_dic:
            gmap_pos_dic[tr] = {chrom+strand:[[start,end]]}
        else:
            if chrom+strand not in gmap_pos_dic[tr]:
                gmap_pos_dic[tr][chrom+strand] = [[start,end]]
            else:
                gmap_pos_dic[tr][chrom+strand].append([start,end])

#### 2. Get gene, assembly, transcripts mapping from pasa descriptions.
Genes can have multiple assebmlies, each assembly maps to many transcripts. 
Here we build an assembly as {gene:{assemble#:[MSTR,XM_**]}}

In [4]:
# get {gene:{assemble:[MSTG,XM_****]}}
# {assmbl:gene}
gene_assem_tr_dic = {}
assm_geneclust_dic = {}
geneclust_chrom_dic = {}
with open(pasa_desp) as f:
    for line in f:
        if line.strip() == '' or line.startswith('#'):
            continue
        item = line.strip().split('\t')
        chrom = item[0]
        gene  = item[1]
        rna  = item[2]
        geneclust_chrom_dic[gene] = chrom
        assm_geneclust_dic[rna] = gene
        trs   = [tr if tr.startswith('MSTRG') else tr.split('.')[0] for tr in item[3].split(',')]
        if gene not in gene_assem_tr_dic:
            gene_assem_tr_dic[gene] = {rna:trs}
        else:
            if rna not in gene_assem_tr_dic[gene]:
                gene_assem_tr_dic[gene][rna] = trs
            else:
                gene_assem_tr_dic[gene][rna].extend(trs)

#### 3. PASA assembly splice positions
{assembly:[chr+strand,[s1,e1],[s2,e2]}

In [5]:
pasa_pos_dic = {}
with open(pasa_gff) as f:
    for line in f:
        if line.strip() == '' or line.startswith('#'):
            continue
        item = line.strip().split('\t')
        chrom = item[0]
        start = int(item[3])
        end = int(item[4])
        strand = item[6]
        anno = item[8]
        tr = re.search('(?<=Target=).+?(?=\ )',anno).group(0)
        if tr not in pasa_pos_dic:
            pasa_pos_dic[tr] = [chrom+strand,[start,end]]
        else:
            pasa_pos_dic[tr].append([start,end])

#### 4. Exonerate protein position

In [6]:
exonerate_pos_dic = {}
pre_pattern = ''
with open(exonerate_gff) as f:
    for line in f:
        if line.strip() == '' or line.startswith('#'):
            continue
        item = line.strip().split('\t')
        chrom = item[0]
        start = int(item[3])
        end = int(item[4])
        strand = item[6]
        anno = item[8]
        pr = re.search('(?<=Target=).+?(?=\.)',anno).group(0)
        pattern = chrom + strand + pr
        if pr not in exonerate_pos_dic:
            exonerate_pos_dic[pr] = {chrom+strand:[[start,end]]}
        else:
            if chrom+strand not in exonerate_pos_dic[pr]:
                exonerate_pos_dic[pr][chrom+strand]=[[start,end]]
            else:
                exonerate_pos_dic[pr][chrom+strand].append([start,end])

#### 5. build annotation 

In [7]:
# build gene, traccess, pr access mapping
from Bio import Entrez
Entrez.email = 'shl198@eng.ucsd.edu'
def gene_rna_pr_id_map(hamster_id,access,out_fn):
    '''this function gets all gene rna pr id, including both refseq and gff information.
    * hamster_id: a file that has all ids in hamster.gff file
    * access: a list accessions
    * out_fn:  
    '''
    # rna accession in gff file
    ham_id_df = pd.read_csv(hamster_id,sep='\t',header=0)
    ham_id_df = ham_id_df.astype('str')
    ham_id_df['TrAccess'] = ham_id_df['TrAccess'].map(lambda x: x.split('.')[0])
    ham_id_df['PrAccess'] = ham_id_df['PrAccess'].map(lambda x: x.split('.')[0])
    rna_gene_dic = ham_id_df.set_index('TrAccess')['GeneID'].to_dict()
    rna_pr_dic = ham_id_df.set_index('TrAccess')['PrAccess'].to_dict()
    rna_gname_dic = ham_id_df.set_index('TrAccess')['GeneSymbol'].to_dict()
    # new rna in refseq compared to gff
    new_ref_rna = list(set(access) - set(rna_gene_dic.keys()))
    # get geneid for new ref_rna gene id
    for r in new_ref_rna:
        if r.startswith('MSTRG'):
            rna_gene_dic[r] = '.'.join(r.split('.')[:2])
            rna_pr_dic[r] = '-'
            rna_gname_dic[r] = '.'.join(r.split('.')[:2])
        else:
            handle = Entrez.efetch(db='nucleotide',id=r,rettype='gb',retmode='text').read()
            
            geneid = re.search('(?<=GeneID:).+?(?=\")',handle).group(0)
            gename = re.search('(?<=gene=\").+?(?=\")',handle).group(0)
            try:
                p = re.search('(?<=protein_id=\").+?(?=\.)',handle).group(0)
            except:
                p = '-'
            rna_gene_dic[r] = geneid
            rna_pr_dic[r] = p
            rna_gname_dic[r] = gename
    # transfer dic to dataframe
    r_g_df = pd.DataFrame.from_dict(rna_gene_dic,'index')
    r_g_df.columns = ['geneid']
    r_p_df = pd.DataFrame.from_dict(rna_pr_dic,'index')
    r_p_df.columns = ['pr_ac']
    r_n_df = pd.DataFrame.from_dict(rna_gname_dic,'index')
    r_n_df.columns = ['gename']
    g_r_p_df = pd.concat([r_g_df,r_n_df,r_p_df],axis=1)
    g_r_p_df['rna_ac'] = g_r_p_df.index
    g_r_p_df[['geneid','gename','rna_ac','pr_ac']].to_csv(out_fn,sep='\t',index=False)

In [7]:
#------------- get gene, tr, pr id mapping
all_trs = []
for k,v in gene_assem_tr_dic.iteritems():
    for k1,v1 in v.iteritems():
        all_trs.extend(v1)
all_trs = list(set(all_trs))
if not os.path.exists(id_map_fn):
    gene_rna_pr_id_map(hamster_id,all_trs,id_map_fn)

#### 6. compare pasa assebly with gmap results

In [8]:
id_df = pd.read_csv(id_map_fn,sep='\t',header=0)
tr_gid_dic = id_df.set_index('rna_ac')['geneid'].to_dict()
tr_pr_dic = id_df.set_index('rna_ac')['pr_ac'].to_dict()
tr_gnm_dic = id_df.set_index('rna_ac')['gename'].to_dict()
tr_gnm_dic = {k:v.upper() for k,v in tr_gnm_dic.iteritems()}
pr_tr_dic = id_df.set_index('pr_ac')['rna_ac'].to_dict()
known_genes = list(set([str(g) for g in id_df['geneid'] if not g.startswith('MSTRG.')]))

In [9]:
assm_refseq_dic = {}  # stores {assemble_id: [refseq_tr]} that have exact same splice
for gene,assm_dic in gene_assem_tr_dic.iteritems():
    for assm,rnas in assm_dic.iteritems(): # {assem:[rnas]}
        assm_chr = pasa_pos_dic[assm][0]
        assm_pos = natsorted([p for pos in pasa_pos_dic[assm][1:] for p in pos])
        for rna in rnas:
            if rna in gmap_pos_dic:
                for rna_k,rna_v in gmap_pos_dic[rna].iteritems(): #{chr+str:[[]]}
                    refseq_pos = natsorted([p for pos in rna_v for p in pos])
                if [assm_chr,assm_pos] == [rna_k,refseq_pos]:
                        if assm not in assm_refseq_dic:
                            assm_refseq_dic[assm] = [rna]
                        else:
                            assm_refseq_dic[assm].append(rna)

In [10]:
print('There are '+ str(len(pasa_pos_dic)) + ' transcript in pasa assembly')
print('Among them ' +str(len(assm_refseq_dic))+' have the same position with gmap')

There are 78290 transcript in pasa assembly
Among them 25321 have the same position with gmap


#### 7. compare gmap consistent  assembly with exonerate

In [11]:
n = 0
assm_pr_dic = {}  # {assemble_id:[pr accessions]}
pr_pos_dic = {}   # {praccession:[chr+str,[pos]}
for assm,ref_ac in assm_refseq_dic.iteritems(): # k is assmble_id, v is list of refseq accession
    prs = [tr_pr_dic[t] for t in ref_ac]
    assm_chr = pasa_pos_dic[assm][0]
    assm_pos = natsorted([p for pos in pasa_pos_dic[assm][1:] for p in pos])
    for pr in prs:
        if pr in exonerate_pos_dic:
            for pr_k,pr_v in exonerate_pos_dic[pr].iteritems(): #{chr+str:[[pos]]}
                pr_pos = natsorted([p for pos in pr_v for p in pos])
                if pr_k == assm_chr: # chromosome and strand consistent
                    if len(pr_pos) == 2: # one CDS protein
                        if len(assm_pos) == 2:
                            if assm_pos[0] <= pr_pos[0]<=pr_pos[1]<=assm_pos[1]:
                                add = True
                            else:
                                add = False
                        else:
                            n += 1
                            add = False #assert False,assm_pos#'one CDS in two exon rna'
                            for pp in pasa_pos_dic[assm][1:]:
                                if pp[0]<=pr_pos[0]<=pr_pos[1]<=pp[1]:
                                    add=True
                    else: # more than one CDS
                        if len(set(pr_pos[1:-1]).intersection(assm_pos)) == len(pr_pos[1:-1]):
                            add = True
                        else:
                            add = False
                else:
                    add = False
                if add == True:
                    pr_pos_dic[pr] = [pr_k]+pr_v
                    if not assm in assm_pr_dic:
                        assm_pr_dic[assm] = [pr]
                    else:
                        assm_pr_dic[assm].append(pr)
print('there are '+str(n)+' protiens that have one CDS but corresponding rna has more than one exon')

there are 626 protiens that have one CDS but corresponding rna has more than one exon


In [12]:
# keep refseq rna accession whose protein splices maps perfect with pasa rna assmblies
assm_tr_dic = {}
for assm,prs in assm_pr_dic.iteritems():  # {assemble_id:[praccessions]}
    trs = [pr_tr_dic[p] for p in prs]
    assm_tr_dic[assm] = trs

In [13]:
tr_assm_dic = {v[0]:k for k,v in assm_tr_dic.iteritems()}

** Till now all transcripts assmblies position is in pasa_pos_dic, all protein position is in pr_pos_dic, all assmblies to tr/pr ids are in assm_tr_dic,assm_pr_dic. **

#### 8. build transdecoder and pr position
{cds_pr_id:[chrom+strand,[s,e],...]}

In [14]:
trans_pr_pos_dic = {}
with open(transde_gff) as f:
    for line in f:
        if line.strip() == '': continue
        item = line.split('\t')
        chrom = item[0]
        start = int(item[3])
        end = int(item[4])
        strand = item[6]
        anno = item[8]
        if item[2] =='CDS':
            fid = re.search('(?<=ID=).+?(?=;)',anno).group(0)[4:]
            if fid not in trans_pr_pos_dic:
                trans_pr_pos_dic[fid] = [chrom+strand,[start,end]]
            else:
                trans_pr_pos_dic[fid].append([start,end])

#### 9. extract all protein sequence from transdecoder that are not verified by previous steps.
** 1). These proteins don't have exact splice sites with pasa assemblies.
Next, we blast protein sequence of those un-verified transcripts to hamster protein again to see if any of them maps perfectly to refseq proteins, this means the exonerate cannot map proteins correctly. **

In [15]:
lncrna = [k for k,v in assm_refseq_dic.iteritems() if v[0].startswith('XR_')]
non_refseq_asmbl = [k for k in pasa_pos_dic if k not in assm_tr_dic and k not in lncrna]
print('there are ' +str(len(non_refseq_asmbl))+' pasa assemblies need to be further verified')

there are 56835 pasa assemblies need to be further verified


In [25]:
# get longest protien for transdecoder proteins.
longest_pr = {}  # {assm:cds_assm}
with open(transde_cds) as f:
    for line in f:
        if line.startswith('>'):
            cds_assm = re.search('(?<=>).+?(?=\ )',line).group(0)
            if cds_assm not in trans_pr_pos_dic:
                continue
            assm = cds_assm.split('|')[0]
            length = int(re.search('(?<=len:).+?(?=\ )',line).group(0))
            if assm not in longest_pr:
                longest_pr[assm] = [cds_assm,length]
            else:
                if length > longest_pr[assm][1]:
                    longest_pr[assm] = [cds_assm,length]
for k in longest_pr:
    longest_pr[k] = longest_pr[k][0]
print('there are '+str(len(longest_pr))+' assemblies have predicted proteins')

there are 57484 assemblies have predicted proteins


In [16]:
# extract proteins that need to be blasted to uniprot and hamster refseq.
trans_pr_fn = pasa_path + '/trans_pr.fa'
if not os.path.exists(trans_pr_fn):
    with open(transde_cds) as in_h,open(trans_pr_fn,'w') as out_h:
        for record in SeqIO.parse(in_h,'fasta'):
            rid = record.id
            assm_id = rid.split('|')[0]
            if assm_id in non_refseq_asmbl and rid in longest_pr.values():
                SeqIO.write(record,out_h,'fasta')
# blast
'''blastp -db hamster -query trans_pr.fa -out map2hamster.txt -outfmt
'6 qacc sacc pident length mismatch gapopen qstart qend sstart send evalue qlen slen'
-num_alignments 1 -num_threads 3 -evalue 1e-7'''

"blastp -db hamster -query trans_pr.fa -out map2hamster.txt -outfmt\n'6 qacc sacc pident length mismatch gapopen qstart qend sstart send evalue qlen slen'\n-num_alignments 1 -num_threads 3 -evalue 1e-7"

In [18]:
# parse blast results of transdecoder proteins to hamster refseq
trans_blast2ham_fn = pasa_path + '/trans_map2hamster.txt'
trans_blast2ham_df = pd.read_csv(trans_blast2ham_fn,sep='\t',header=None,names=['qacc',
        'sacc','pident','length','mismatch','gapopen','qstart','qend','sstart','send',
        'evalue','qlen','slen'])
trans_blast2ham_df['assm'] = trans_blast2ham_df['qacc'].map(lambda x: x.split('|')[0])
trans_blast2ham_df['sacc'] = trans_blast2ham_df['sacc'].map(lambda x: x.split('.')[0])

# trans_blast2ham_df[trans_blast2ham_df['qacc'].isin(longest_pr.values())
#                   ].iloc[:,range(14)].to_csv(trans_blast2ham_fn,sep='\t',header=None,index=False)

In [19]:
# get perfect mapping
blt2ham100map_df = trans_blast2ham_df.query('(pident==100) and (length==qlen-1)')
# cri = blt2ham100map_df[['assm','sacc','length']].groupby(['assm'])['length'].transform(max) == blt2ham100map_df['length']
# blt2ham100map_df = blt2ham100map_df[cri]
print('there are '+str(blt2ham100map_df['assm'].unique().shape[0])+' assemblies whose protein map perfectly to refseq')
# stores the assemblies that have perfect blastp mapping to refseq 
# but missing by previous steps
n = 0
assm_blt2ham_tr_pr100map_dic = {} 
for assm,pr,cds in zip(blt2ham100map_df['assm'],blt2ham100map_df['sacc'],blt2ham100map_df['qacc']):
    if assm in assm_tr_dic: #or ([pr] in assm_pr_dic.values()):
        n += 1
        print assm
    assm_blt2ham_tr_pr100map_dic[assm] = [pr,cds]

there are 12521 assemblies whose protein map perfectly to refseq


** 2). After getting mappings for pasa exonerate matched proteins and non coding rnas, then we get proteins that map more than 80% of the whole sequence with at least 80% identify. **

In [21]:
# assemblies not identified by previous steps
non_refseq_asmbl1 = [assm for assm in non_refseq_asmbl if assm not in assm_blt2ham_tr_pr100map_dic]
print len(non_refseq_asmbl1),'assemblies do not map to know proteins'

44314 assemblies do not map to know proteins


In [188]:
# pasa_pr_consist_genes = list(set([assm_geneclust_dic[assm] for assm in assm_tr_dic.keys()+\
#                                  assm_blt2ham_tr_pr100map_dic.keys()]))
# # use previous identified genes to recover assemblies not included in previous steps 
# # these rescued genes will have entrez gene id and names
# gene_rescued_assm = []
# for assm in non_refseq_asmbl1:
#     if assm_geneclust_dic[assm] in pasa_pr_consist_genes:
#         gene_rescued_assm.append(assm)

# # assmblies left
# non_refseq_asmbl2 = list(set(non_refseq_asmbl1) - set(gene_rescued_assm))

In [22]:
blt2ham80map_df = trans_blast2ham_df[trans_blast2ham_df['assm'].isin(non_refseq_asmbl1) & 
                  (trans_blast2ham_df['pident'].values>80) &
                  (trans_blast2ham_df['pident'].values<100) & 
                   (trans_blast2ham_df['length']/trans_blast2ham_df['qlen'] > 0.8)]
assm_blt2ham_tr_pr80map_dic = {}
for assm,pr,cds in zip(blt2ham80map_df['assm'],blt2ham80map_df['sacc'],blt2ham80map_df['qacc']):
    if assm not in assm_blt2ham_tr_pr80map_dic:
        assm_blt2ham_tr_pr80map_dic[assm] = [pr,cds]
    else:
        if assm_blt2ham_tr_pr80map_dic[assm] != [pr,cds]:
            print assm

In [23]:
non_refseq_asmbl2 = list(set(non_refseq_asmbl1) - set(assm_blt2ham_tr_pr80map_dic))
print len(non_refseq_asmbl2),'assemblies do not map to know proteins'

30706 assemblies do not map to know proteins


** 3). For the rest of the assemblies, we blast to uniprot to decide their functions **

In [26]:
trans_blast2uni_fn = pasa_path + '/trans_map2uniprot.txt'
trans_blast2uni_df = pd.read_csv(trans_blast2uni_fn,sep='\t',header=None,names=['qacc',
        'sacc','pident','length','mismatch','gapopen','qstart','qend','sstart','send',
        'evalue','bit','qlen','slen'])
trans_blast2uni_df['assm'] = trans_blast2uni_df['qacc'].map(lambda x: x.split('|')[0])
trans_blast2uni_df = trans_blast2uni_df[(trans_blast2uni_df['assm'].isin(non_refseq_asmbl2)) & 
                                       (trans_blast2uni_df['qacc'].isin(longest_pr.values()))]
trans_blast2uni_df['gename'] = trans_blast2uni_df['sacc'].map(lambda x: 
                                        (x.split('|')[2]).split('_')[0])

# trans_blast2uni_df = pd.read_csv(trans_blast2uni_fn,sep='\t',header=None)
# trans_blast2uni_df['qlen'] = trans_blast2uni_df[0].map(lambda x: len(hamster_index[x].seq))
# trans_blast2uni_df['slen'] = trans_blast2uni_df[1].map(lambda x: len(uniprot_index[x].seq))
# trans_blast2uni_df.to_csv(trans_blast2uni_fn,sep='\t',header=None,index=False)

In [545]:
# trans_blast2uni_df = trans_blast2uni_df[(trans_blast2uni_df['pident'].values>60) & 
#                   (trans_blast2uni_df['length']/trans_blast2uni_df['qlen']>0.6)]

In [27]:
assm_gename_dic = {k:list(set(v)) for k,v in trans_blast2uni_df.groupby('assm')['gename']}
print('There are '+str(len(assm_gename_dic))+' pr coding genes verified by uniprot')

There are 9776 pr coding genes verified by uniprot


In [28]:
# build {gene:assembly} dictionary
gename_assm_dic = {}
for k,v in assm_gename_dic.iteritems():
    if v[0] in gename_assm_dic:
        gename_assm_dic[v[0]].append(k)
    else:
        gename_assm_dic[v[0]] = [k]

In [29]:
non_refseq_asmbl3 = list(set(non_refseq_asmbl2) - set(assm_gename_dic.keys()))
print('there are '+str(len(non_refseq_asmbl3))+' assemblies left')

there are 20930 assemblies left


#### 10. build gene cluster gene id, gene name dic
1. assm_tr_dic (16679). geneid, genename, tr_ac,pr_ac
2. assm_blt2ham_tr_pr100map_dic (9693), geneid,genename,assm_id,pr_ac
3. gene_rescued_assm (17898). geneid,genename,assm_id,cds_assm
4. assm_gename_dic (12804): gene_clust,genename,assm_id,cds_assm

In [30]:
def update_gene_clust_idname_dic(gene_clust_idname_dic,gclust,gid,gnm,assm,known_genes):
    if gclust not in gene_clust_idname_dic:
        gene_clust_idname_dic[gclust] = [[gid,gnm,assm]]
    else:
        gnms = [g[1] for g in gene_clust_idname_dic[gclust]]
        gids = [g[0] for g in gene_clust_idname_dic[gclust]]
        if gnm not in gnms:
            gene_clust_idname_dic[gclust].append([gid,gnm,assm])
#             if gid in known_genes:
#                 gene_clust_idname_dic[gclust].append([gid,gnm,assm])
#             else:  
#             # here we discard some assemblies that can't be decided to known or novel
#             # need to think more, I think i should include all gene_# ids.
#                 if len(gids)==1:
#                     gene_clust_idname_dic[gclust][0].append(assm)
        else:
            index = gnms.index(gnm)
            gene_clust_idname_dic[gclust][index].append(assm)
    return gene_clust_idname_dic

# build gene cluster
gene_clust_idname_dic = {}
for assm in assm_tr_dic: # case 1
    gclust = assm_geneclust_dic[assm]
    gid = tr_gid_dic[assm_tr_dic[assm][0]]
    gnm = tr_gnm_dic[assm_tr_dic[assm][0]]
    gene_clust_idname_dic = update_gene_clust_idname_dic(gene_clust_idname_dic,gclust,gid,gnm,assm,known_genes)
for assm in assm_blt2ham_tr_pr100map_dic: # case 2
    gclust = assm_geneclust_dic[assm]
    pr = assm_blt2ham_tr_pr100map_dic[assm][0]
    gid = tr_gid_dic[pr_tr_dic[pr]]
    gnm = tr_gnm_dic[pr_tr_dic[pr]]
    gene_clust_idname_dic = update_gene_clust_idname_dic(gene_clust_idname_dic,gclust,gid,gnm,assm,known_genes)
for assm in assm_blt2ham_tr_pr80map_dic: # case 2
    gclust = assm_geneclust_dic[assm]
    pr = assm_blt2ham_tr_pr80map_dic[assm][0]
    gid = tr_gid_dic[pr_tr_dic[pr]]
    gnm = tr_gnm_dic[pr_tr_dic[pr]]
    gene_clust_idname_dic = update_gene_clust_idname_dic(gene_clust_idname_dic,gclust,gid,gnm,assm,known_genes)
for assm in assm_gename_dic:
    gclust = assm_geneclust_dic[assm]
    gid = 'gene_' + assm_geneclust_dic[assm]
    gnm = assm_gename_dic[assm][0]
    gene_clust_idname_dic = update_gene_clust_idname_dic(gene_clust_idname_dic,gclust,gid,gnm,assm,known_genes)
for gclust in gene_assem_tr_dic:
    if gclust not in gene_clust_idname_dic:
        assms = gene_assem_tr_dic[gclust].keys()
        gene_clust_idname_dic[gclust] = [['gene_'+gclust]*2+assms]

#### 11. output to gff

In [31]:
def get_pasa_pos_info(pasa_pos_dic,assm):
    tr_pos = pasa_pos_dic[assm]
    tr_str = tr_pos[0][-1]
    tr_chrom = tr_pos[0][:-1]
    tr_stt = str(tr_pos[1][0])
    tr_end = str(tr_pos[-1][-1])
    tr_pos = tr_pos[1:]
    if tr_str == '-':
        tr_pos = sorted(tr_pos,key=lambda x:x[0],reverse=True)
    return [tr_chrom,tr_stt,tr_end,tr_str,tr_pos]

def get_exonerate_pos_info(pr_pos_dic,pr_ac):
    pr_pos = pr_pos_dic[pr_ac]
    pr_str = pr_pos[0][-1]
    pr_chrom = pr_pos[0][:-1]
    pr_stt = str(pr_pos[1][0])
    pr_end = str(pr_pos[-1][-1])
    if pr_str == '-':
        pr_stt, pr_end = pr_end,pr_stt
    return [pr_chrom,pr_stt,pr_end,pr_str,pr_pos[1:]]

def get_trans_cds_pos_info(trans_pr_pos_dic,pr_id):
    pr_pos = trans_pr_pos_dic[pr_id]
    pr_str = pr_pos[0][-1]
    pr_chrom = pr_pos[0][:-1]
    pr_stt = str(pr_pos[1][0])
    pr_end = str(pr_pos[-1][-1])
    if pr_str == '-':
        pr_stt, pr_end = pr_end,pr_stt
    return [pr_chrom,pr_stt,pr_end,pr_str,pr_pos[1:]]

In [32]:
with open(pasa_path +'/merge.gff3','w') as gff:
    for gene in natsorted(gene_assem_tr_dic.keys()):
        # get assembly id
        chrom = geneclust_chrom_dic[gene]
        if len(gene_clust_idname_dic[gene]) == 1: # one gene cluster maps to only one gene
            try:
                [geneid,gename] = gene_clust_idname_dic[gene][0][:2]
            except:
                geneid = gename = 'gene_'+gene
            assm_dic = gene_assem_tr_dic[gene]  # {assm:[tr]}
            out_assms = [[geneid,gename]+assm_dic.keys()]
        else:
            out_assms = gene_clust_idname_dic[gene] # [[geneid,genename,assm_id1,assm_id2,..]]
        for assms in out_assms:
            [geneid,gename] = assms[:2]
            for assm in natsorted(assms[2:]):
                if assm in assm_tr_dic:
                    # get tr information
                    tr_ac  = assm_tr_dic[assm][0]
                    # get pr information
                    pr_ac  = assm_pr_dic[assm][0]
                    pr_info = get_exonerate_pos_info(pr_pos_dic,pr_ac)
                    cr = 1
                elif assm in assm_blt2ham_tr_pr100map_dic or assm in assm_blt2ham_tr_pr80map_dic:
                    try:
                        [pr_ac,pr_id] = assm_blt2ham_tr_pr100map_dic[assm]
                    except:
                        [pr_ac,pr_id] = assm_blt2ham_tr_pr80map_dic[assm]
                    tr_ac = pr_tr_dic[pr_ac]
                    gid = tr_gid_dic[tr_ac]
                    if gid  != geneid:
                        print assm
                    pr_info = get_trans_cds_pos_info(trans_pr_pos_dic,pr_id)
                    cr = 2
                elif assm in assm_gename_dic: # this is got from mapping to swiss prot
                    pr_ac = longest_pr[assm]
                    pr_info = get_trans_cds_pos_info(trans_pr_pos_dic,pr_ac)
                    tr_ac = assm
                    cr = 3
                elif assm in lncrna:
                    tr_ac = assm_refseq_dic[assm][0]
                    pr_ac = ''
                    cr = 1
                else:
                    tr_ac = assm
                    try:
                        pr_ac = longest_pr[assm]
                        pr_info = get_trans_cds_pos_info(trans_pr_pos_dic,pr_ac)
                    except:
                        pr_ac = ''
                    cr = 4
                tr_info = get_pasa_pos_info(pasa_pos_dic,assm)
                # sometimes transdecoder and pasa assemblies have different strand, use transdecoder
                if (pr_info[3] != tr_info[3] or pr_info[0]!= tr_info[0]) and pr_ac !='':
                    tr_info[3] = pr_info[3]
                    print(assm+' tr and pr are not in the same scaffold/strand')
                # write mRNA line
                rna_anno = ('ID={assm};Parent={gid};gene_id={gid};Name={gnm};'
                'transcript_id={tr};cr={cr}').format(assm=assm,gid=geneid,gnm=gename,tr=tr_ac,cr=cr)
                if geneid == gename: # 
                    feature = 'mRNA'
                    gff.write('\t'.join([tr_info[0],'merge',feature,tr_info[1],tr_info[2],'.',tr_info[3],
                                     '.',rna_anno+';gene unknown'])+'\n')
                else:
                    if assm in lncrna:
                        feature = 'lncRNA'
                    else:
                        feature = 'mRNA'
                    gff.write('\t'.join([tr_info[0],'merge',feature,tr_info[1],tr_info[2],'.',tr_info[3],
                                         '.',rna_anno])+'\n')
                n = 0
                for t in tr_info[-1]:
                    n +=1
                    tr_anno = ('ID={assm}.exon{n};Parent={assm};gene_id={gid};Name={gnm};'
                        'transcript_id={tr}').format(assm=assm,n=str(n),gid=geneid,gnm=gename,tr=tr_ac)
                    gff.write('\t'.join([tr_info[0],'merge','exon',str(t[0]),str(t[1]),\
                                        '.',tr_info[3],'.',tr_anno])+'\n')
                if pr_ac != '':
                    n = 0
                    for p in pr_info[-1]:
                        n+=1
                        pr_anno = ('ID={assm}.cds{n};Parent={assm};gene_id={gid};Name={gnm};'
                            'protein_id={pr}').format(assm=assm,n=str(n),gid=geneid,gnm=gename,pr=pr_ac)
                        gff.write('\t'.join([pr_info[0],'merge','CDS',str(p[0]),str(p[1]),\
                                             '.',pr_info[3],'.',pr_anno])+'\n')      

asmbl_130 tr and pr are not in the same scaffold/strand
asmbl_173 tr and pr are not in the same scaffold/strand
asmbl_348 tr and pr are not in the same scaffold/strand
asmbl_509 tr and pr are not in the same scaffold/strand
asmbl_586 tr and pr are not in the same scaffold/strand
asmbl_1360 tr and pr are not in the same scaffold/strand
asmbl_2013 tr and pr are not in the same scaffold/strand
asmbl_2304 tr and pr are not in the same scaffold/strand
asmbl_2903 tr and pr are not in the same scaffold/strand
asmbl_2964 tr and pr are not in the same scaffold/strand
asmbl_3025 tr and pr are not in the same scaffold/strand
asmbl_3287 tr and pr are not in the same scaffold/strand
asmbl_3288 tr and pr are not in the same scaffold/strand
asmbl_3559 tr and pr are not in the same scaffold/strand
asmbl_3788 tr and pr are not in the same scaffold/strand
asmbl_3825 tr and pr are not in the same scaffold/strand
asmbl_3840 tr and pr are not in the same scaffold/strand
asmbl_3882 tr and pr are not in the 

asmbl_21097 tr and pr are not in the same scaffold/strand
asmbl_21104 tr and pr are not in the same scaffold/strand
asmbl_21268 tr and pr are not in the same scaffold/strand
asmbl_21570 tr and pr are not in the same scaffold/strand
asmbl_21589 tr and pr are not in the same scaffold/strand
asmbl_21641 tr and pr are not in the same scaffold/strand
asmbl_21686 tr and pr are not in the same scaffold/strand
asmbl_21721 tr and pr are not in the same scaffold/strand
asmbl_21796 tr and pr are not in the same scaffold/strand
asmbl_22009 tr and pr are not in the same scaffold/strand
asmbl_22257 tr and pr are not in the same scaffold/strand
asmbl_22680 tr and pr are not in the same scaffold/strand
asmbl_22704 tr and pr are not in the same scaffold/strand
asmbl_23052 tr and pr are not in the same scaffold/strand
asmbl_23468 tr and pr are not in the same scaffold/strand
asmbl_23534 tr and pr are not in the same scaffold/strand
asmbl_23551 tr and pr are not in the same scaffold/strand
asmbl_23595 tr

asmbl_43270 tr and pr are not in the same scaffold/strand
asmbl_43511 tr and pr are not in the same scaffold/strand
asmbl_43777 tr and pr are not in the same scaffold/strand
asmbl_44127 tr and pr are not in the same scaffold/strand
asmbl_44148 tr and pr are not in the same scaffold/strand
asmbl_44309 tr and pr are not in the same scaffold/strand
asmbl_44316 tr and pr are not in the same scaffold/strand
asmbl_44502 tr and pr are not in the same scaffold/strand
asmbl_44959 tr and pr are not in the same scaffold/strand
asmbl_45325 tr and pr are not in the same scaffold/strand
asmbl_45502 tr and pr are not in the same scaffold/strand
asmbl_45504 tr and pr are not in the same scaffold/strand
asmbl_45626 tr and pr are not in the same scaffold/strand
asmbl_45845 tr and pr are not in the same scaffold/strand
asmbl_46162 tr and pr are not in the same scaffold/strand
asmbl_46182 tr and pr are not in the same scaffold/strand
asmbl_46515 tr and pr are not in the same scaffold/strand
asmbl_46545 tr

asmbl_66164 tr and pr are not in the same scaffold/strand
asmbl_66194 tr and pr are not in the same scaffold/strand
asmbl_66382 tr and pr are not in the same scaffold/strand
asmbl_66457 tr and pr are not in the same scaffold/strand
asmbl_66476 tr and pr are not in the same scaffold/strand
asmbl_66483 tr and pr are not in the same scaffold/strand
asmbl_66515 tr and pr are not in the same scaffold/strand
asmbl_66725 tr and pr are not in the same scaffold/strand
asmbl_66789 tr and pr are not in the same scaffold/strand
asmbl_66791 tr and pr are not in the same scaffold/strand
asmbl_66832 tr and pr are not in the same scaffold/strand
asmbl_67146 tr and pr are not in the same scaffold/strand
asmbl_67565 tr and pr are not in the same scaffold/strand
asmbl_67857 tr and pr are not in the same scaffold/strand
asmbl_67918 tr and pr are not in the same scaffold/strand
asmbl_67989 tr and pr are not in the same scaffold/strand
asmbl_68299 tr and pr are not in the same scaffold/strand
asmbl_68332 tr

In [47]:
gene_rna_dic = {}
rna_all_dic = {}
with open(pasa_path+'/merge.gff3',) as in_f:
    for line in in_f:
        item = line.strip().split('\t')
        if item[2] in ['mRNA','lncRNA']:
            rnaid = re.search('(?<=ID=).+?(?=;)',item[8]).group(0)
            rna_all_dic[rnaid] = [item]
            chrom = item[0]
            strand = item[6]
            geneid = re.search('(?<=gene_id=).+?(?=;)',item[8]).group(0)
            gename = re.search('(?<=Name=).+?(?=;|$)',item[8]).group(0)
            if geneid in gene_rna_dic:
                gene_rna_dic[geneid].append([chrom,rnaid,strand,gename,[int(item[3]),int(item[4])]])
            else:
                gene_rna_dic[geneid] = [[chrom,rnaid,strand,gename,[int(item[3]),int(item[4])]]]
        else:
            rna_all_dic[rnaid].append(item)

In [49]:
with open(pasa_path + '/final.gff3','w') as out_f:
    for gene,rnas in gene_rna_dic.iteritems():
        rnas = sorted(rnas,key=lambda x: (x[0],x[4][0]))
        # get the list of number having the same length with rnas, indicating 
        # whether to write the gene batch line. stores in gene_batch
        n = 0
        gene_batch = []
        gene_pos = {}  # {batch#:[start,end]}
        pre_chrom = ''
        pre_name = ''
        pre_start = pre_end = 0
        for rna in rnas: # [chrom,asmbl_id,[start,end]]
            rna_s = rna[4][0]
            rna_e = rna[4][1]
            if rna[0] != pre_chrom or rna[3] != pre_name:
                n += 1
                gene_batch.append(n)
                gene_pos[n] = [rna_s,rna_e]
            else:
                if rna_s > pre_end:
                    n += 1
                    gene_batch.append(n)
                    gene_pos[n] = [rna_s,rna_e]
                else:
                    gene_batch.append(n)
                    gene_pos[n] = [min(rna_s,gene_pos[n][0]),max(rna_e,gene_pos[n][1])]
            pre_start = rna_s
            pre_end = rna_e
            pre_chrom = rna[0]
            pre_name = rna[3]
        pre_b = 0
        if len(set(gene_batch)) > 1:
            for b,rna in zip(gene_batch,rnas):
                if b != pre_b:
                    anno='ID={g}_{b};gene_id={g};Name={name}'.format(g=gene,b=str(b),name=rna[3])
                    out_f.write('\t'.join([rna[0],'merge','gene',str(gene_pos[b][0]),
                                        str(gene_pos[b][1]),'.',rna[2],'.',anno])+'\n')
                    rna_all_info = rna_all_dic[rna[1]]
                    for rna_item in rna_all_info:
                        if rna_item[2] in ['mRNA','lncRNA']:
                            rna_item[8]=re.sub('(?<=Parent=).+?(?=;)',gene+'_'+str(b),rna_item[8])
                        rna_item[8]=re.sub('(?<=gene_id=).+?(?=;)',gene+'_'+str(b),rna_item[8])
                        out_f.write('\t'.join(rna_item)+'\n')
                    pre_b = b
                else:
                    rna_all_info = rna_all_dic[rna[1]]
                    for rna_item in rna_all_info:
                        if rna_item[2] in ['mRNA','lncRNA']:
                            rna_item[8]=re.sub('(?<=Parent=).+?(?=;)',gene+'_'+str(b),rna_item[8])
                        rna_item[8]=re.sub('(?<=gene_id=).+?(?=;)',gene+'_'+str(b),rna_item[8])
                        out_f.write('\t'.join(rna_item)+'\n')
        else:
            anno='ID={g};gene_id={g};Name={name}'.format(g=gene,name=rna[3])
            out_f.write('\t'.join([rnas[0][0],'merge','gene',str(gene_pos[1][0]),
                                str(gene_pos[1][1]),'.',rna[2],'.',anno])+'\n')
            for rna in rnas:
                rna_all_info = rna_all_dic[rna[1]]
                for rna_item in rna_all_info:
#                     if rna_item[2] in ['mRNA','lncRNA']:
#                         rna_item[8]=re.sub('(?<=Parent=).+?(?=;)',gene,rna_item[8])
                    out_f.write('\t'.join(rna_item)+'\n')

Next we need to add the frame information for all the CDS lines.

In [50]:
# add frame information for CDS
with open(pasa_path+'/pasa_stringtie.gff3','w') as out, open(pasa_path+'/final.gff3') as inf:
    pre_pr = ''
    for line in inf:
        if line.startswith('#'):
            out.write(line)
        else:
            item = line.split('\t')
            if item[2] == 'CDS':
                pr = re.search('(?<=protein_id=).+?(?=$|;)',item[8]).group(0)
                if pr != pre_pr:
                    pre_pr = pr
                    item[7] = '0'
                    n = abs(int(item[3])-int(item[4]))+1
                else:
                    item[7] = str(n % 3)
                    n += abs(int(item[3])-int(item[4]))+1
                out.write('\t'.join(item))
            else:
                out.write(line)
                    